# A Four-Panel Plot

This notebook demonstrates the creation of a four-panel plot. We will plot the 300-mb heights and wind speeds at forecast hours 12, 24, 36, and 48 from today's 12Z NAM on Grid 211.

In [1]:
import sys
sys.path.append('/home/decker/lab433')

from synoptic import MODEL

from datetime import datetime, timedelta

import xarray as xr
from metpy.io import GempakGrid
from metpy.units import units
from metpy.plots import ContourPlot, FilledContourPlot, MapPanel, PanelContainer
import metpy.calc as mpcalc

## Get the Data and Generate Contours

In this case, we are plotting the same thing, but at four different times.  To avoid code repetition, we can use a for loop and move the data ingest and contour creation steps into the loop. (We would not use a for loop if we were plotting four different panels all at the same time.)

In [2]:
gem_file_name = MODEL + 'nam/24092512_nam211.gem'
gem_file = GempakGrid(gem_file_name)
init_time = datetime(2024, 9, 25, 12)

FileNotFoundError: [Errno 2] No such file or directory: '/ldmdata/gempak/model/nam/24092512_nam211.gem'

In [ ]:
panel_list = []
for i in range(1, 5):
    fhr = 12*i
    plot_time = init_time + timedelta(hours=fhr)
    
    # Get data for this time
    ht = gem_file.gdxarray(parameter='HGHT', date_time=plot_time, level=300)[0] * units('m')
    u = gem_file.gdxarray(parameter='UREL', date_time=plot_time, level=300)[0] * units('m/s')
    v = gem_file.gdxarray(parameter='VREL', date_time=plot_time, level=300)[0] * units('m/s')

    # Calculation
    speed = mpcalc.wind_speed(u, v)
    
    # Create contours for this time
    isotachs = FilledContourPlot()
    isotachs.data = speed
    isotachs.time = plot_time
    isotachs.contours = range(70,200,20)
    isotachs.colormap = 'autumn_r'
    isotachs.colorbar = 'horizontal'
    isotachs.plot_units = 'knot'
    
    isohypses = ContourPlot()
    isohypses.data = ht
    isohypses.time = plot_time
    isohypses.contours = range(0,1000,12)
    isohypses.linecolor = 'darkgreen'
    isohypses.linestyle = 'solid'
    isohypses.clabels = True
    isohypses.plot_units = 'dam'
    
    # Construct the panel for this time
    panel = MapPanel()
    panel.layout = (2, 2, i)
    panel.area = [-120, -74, 22, 55]
    panel.projection = 'lcc'
    panel.layers = ['states', 'coastline', 'borders']
    panel.title = f'300-mb Heights (dam) and Winds (kt) at {plot_time}'
    panel.plots = [isotachs, isohypses]
    
    panel_list.append(panel)    

## Assemble the panels and show the plot

In [ ]:
pc = PanelContainer()
pc.size = (22, 17)
pc.panels = panel_list
pc.show()

In [ ]:
pc.save('4panel.svg', bbox_inches='tight', dpi=300)